# HDF5 Dataset Creation

In [62]:
# Import libraries
from pathlib import Path
import requests
import json 
import urllib
import logging
from io import BytesIO
from zipfile import ZipFile, BadZipFile
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import pandas_datareader.data as web
from sklearn.datasets import fetch_openml

pd.set_option('display.expand_frame_repr', False)

In [63]:
import warnings
warnings.filterwarnings('ignore')

In [64]:
# Set path for HDF5 file and other datasets returned via this notebook
data_path = Path('../data') 
if not data_path.exists():
    data_path.mkdir()

# Set path for HDF5 Data Store
DATA_STORE = Path('../data/assets.h5')

## Alpaca Data Retrieval

In [65]:
load_dotenv()                                                               # Load .env enviroment variables
alpaca_api_key = os.getenv("ALPACA_API_KEY")                                # Set Alpaca API key from .env
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")                          # Set Alpaca API secret from .env
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')    # Create Alpaca Trade API client

In [66]:
# Tesla Stock Data Retrieval
## Set timeframe to "1Min"
timeframe = "1Min"

# Format current date as ISO format 
start_date = pd.Timestamp("2017-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-10-07", tz="America/New_York").isoformat()

# Set the stock tickers 
tickers = ["TSLA"]

# Retrieve Tesla stock data via Alpaca
df = api.get_bars(
    tickers, 
    timeframe, 
    start = start_date,
    end = end_date,
).df

# Display dataframe info
display(df.info())

# Preview dataframe
display(df.head(3))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 934339 entries, 2017-01-03 09:00:00+00:00 to 2022-10-06 23:59:00+00:00
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   open         934339 non-null  float64
 1   high         934339 non-null  float64
 2   low          934339 non-null  float64
 3   close        934339 non-null  float64
 4   volume       934339 non-null  int64  
 5   trade_count  934339 non-null  int64  
 6   vwap         934339 non-null  float64
 7   symbol       934339 non-null  object 
dtypes: float64(5), int64(2), object(1)
memory usage: 64.2+ MB


None

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2017-01-03 09:00:00+00:00,213.76,214.03,213.76,214.03,1087,8,213.875658,TSLA
2017-01-03 11:35:00+00:00,214.91,214.91,214.91,214.91,109,2,214.905046,TSLA
2017-01-03 12:00:00+00:00,215.22,215.22,215.22,215.22,400,1,215.220000,TSLA


In [67]:
# Save dataframe as csv file
df.to_csv(Path("../data/tesla_prices.csv"))

In [68]:
# Read and store Tesla price data
df = (pd.read_csv('../data/tesla_prices.csv',
                 index_col=['timestamp', 'symbol'],
                 infer_datetime_format=True)
     .sort_index())

# View dataframe info
print(df.info(null_counts=True))

# Store Alpaca dataframe in HDFStore
with pd.HDFStore(DATA_STORE) as store:
    store.put('alpaca/tesla/prices', df)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 934339 entries, ('2017-01-03 09:00:00+00:00', 'TSLA') to ('2022-10-06 23:59:00+00:00', 'TSLA')
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   open         934339 non-null  float64
 1   high         934339 non-null  float64
 2   low          934339 non-null  float64
 3   close        934339 non-null  float64
 4   volume       934339 non-null  int64  
 5   trade_count  934339 non-null  int64  
 6   vwap         934339 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 93.7+ MB
None


In [76]:
# SP500 data retrieval
## Set timeframe to "1Min"
timeframe = "1Min"

# Format current date as ISO format 
start_date = pd.Timestamp("2017-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-10-07", tz="America/New_York").isoformat()

# Set the stock tickers 
tickers = ["SPY"]

# Retrieve Tesla stock data via Alpaca
df = api.get_bars(
    tickers, 
    timeframe, 
    start = start_date,
    end = end_date,
).df

# Display dataframe info
display(df.info())

# Preview dataframe
display(df.head(3))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1155329 entries, 2017-01-03 09:00:00+00:00 to 2022-10-06 23:59:00+00:00
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   open         1155329 non-null  float64
 1   high         1155329 non-null  float64
 2   low          1155329 non-null  float64
 3   close        1155329 non-null  float64
 4   volume       1155329 non-null  int64  
 5   trade_count  1155329 non-null  int64  
 6   vwap         1155329 non-null  float64
 7   symbol       1155329 non-null  object 
dtypes: float64(5), int64(2), object(1)
memory usage: 79.3+ MB


None

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2017-01-03 09:00:00+00:00,224.80,224.83,224.30,224.83,4960,30,224.764929,SPY
2017-01-03 09:01:00+00:00,224.83,224.83,224.60,224.60,5976,6,224.764839,SPY
2017-01-03 09:02:00+00:00,224.77,224.77,224.77,224.77,1994,4,224.770000,SPY


In [77]:
# Save dataframe as csv file
df.to_csv(Path("../data/sp500_prices.csv"))

In [78]:
# Read and store s&p500 price data
df = (pd.read_csv('../data/SP500_prices.csv',
                 index_col=['timestamp', 'symbol'],
                 infer_datetime_format=True)
     .sort_index())

# View dataframe info
print(df.info(null_counts=True))

# Store Alpaca dataframe in HDFStore
with pd.HDFStore(DATA_STORE) as store:
    store.put('alpaca/SP500/prices', df)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1155329 entries, ('2017-01-03 09:00:00+00:00', 'SPY') to ('2022-10-06 23:59:00+00:00', 'SPY')
Data columns (total 7 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   open         1155329 non-null  float64
 1   high         1155329 non-null  float64
 2   low          1155329 non-null  float64
 3   close        1155329 non-null  float64
 4   volume       1155329 non-null  int64  
 5   trade_count  1155329 non-null  int64  
 6   vwap         1155329 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 108.3+ MB
None


## QUANDL Data Retrieval

In [79]:
# Error logging
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)  
data_link_log = logging.getLogger("nasdaqdatalink")
data_link_log.setLevel(logging.DEBUG)

In [80]:
# API Endpoint
url = "https://www.quandl.com/api/v3/datasets/WIKI/TSLA.json?api_key=XmfpCC9s2CV5sed4dH4z" #TODO: Add docstring re: API key 
response = requests.get(url)
print(response)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.quandl.com:443
DEBUG:urllib3.connectionpool:https://www.quandl.com:443 "GET /api/v3/datasets/WIKI/TSLA.json?api_key=XmfpCC9s2CV5sed4dH4z HTTP/1.1" 200 None


<Response [200]>


In [81]:
# Extract content of API response
content = response.content

# Convert output to json 
data = response.json()

# Format data with json dumps
print(json.dumps(data, indent=4, sort_keys=True))

{
    "dataset": {
        "collapse": null,
        "column_index": null,
        "column_names": [
            "Date",
            "Open",
            "High",
            "Low",
            "Close",
            "Volume",
            "Ex-Dividend",
            "Split Ratio",
            "Adj. Open",
            "Adj. High",
            "Adj. Low",
            "Adj. Close",
            "Adj. Volume"
        ],
        "data": [
            [
                "2018-03-27",
                304.0,
                304.27,
                277.18,
                279.18,
                13696168.0,
                0.0,
                1.0,
                304.0,
                304.27,
                277.18,
                279.18,
                13696168.0
            ],
            [
                "2018-03-26",
                307.34,
                307.59,
                291.36,
                304.18,
                8324639.0,
                0.0,
                1.0,
             

In [82]:
# Convert output to pandas dataframe
df = pd.DataFrame(data['dataset']['data'], columns=data['dataset']['column_names'])
df.set_index('Date', inplace=True)
print(df.head())

              Open    High     Low   Close      Volume  Ex-Dividend  Split Ratio  Adj. Open  Adj. High  Adj. Low  Adj. Close  Adj. Volume
Date                                                                                                                                     
2018-03-27  304.00  304.27  277.18  279.18  13696168.0          0.0          1.0     304.00     304.27    277.18      279.18   13696168.0
2018-03-26  307.34  307.59  291.36  304.18   8324639.0          0.0          1.0     307.34     307.59    291.36      304.18    8324639.0
2018-03-23  311.25  311.61  300.45  301.54   6600538.0          0.0          1.0     311.25     311.61    300.45      301.54    6600538.0
2018-03-22  313.89  318.82  308.18  309.10   4914307.0          0.0          1.0     313.89     318.82    308.18      309.10    4914307.0
2018-03-21  310.25  322.44  310.19  316.53   5927881.0          0.0          1.0     310.25     322.44    310.19      316.53    5927881.0


In [83]:
# Pass 'Date' colum in as date format
df.index = pd.to_datetime(df.index)
df.head(3)

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2018-03-27,304.00,304.27,277.18,279.18,13696168.0,0.0,1.0,304.00,304.27,277.18,279.18,13696168.0
2018-03-26,307.34,307.59,291.36,304.18,8324639.0,0.0,1.0,307.34,307.59,291.36,304.18,8324639.0
2018-03-23,311.25,311.61,300.45,301.54,6600538.0,0.0,1.0,311.25,311.61,300.45,301.54,6600538.0


In [84]:
# Save dataframe as csv
df.to_csv('../data/quandl.csv')

In [85]:
# Read in csv file
df = (pd.read_csv('../data/quandl.csv').sort_index())

# Display dataframe info
print(df.info(null_counts=True))

# Store df in HDFStore
with pd.HDFStore(DATA_STORE) as store:
    store.put('quandl/wiki/prices', df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1949 entries, 0 to 1948
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         1949 non-null   object 
 1   Open         1949 non-null   float64
 2   High         1949 non-null   float64
 3   Low          1949 non-null   float64
 4   Close        1949 non-null   float64
 5   Volume       1949 non-null   float64
 6   Ex-Dividend  1949 non-null   float64
 7   Split Ratio  1949 non-null   float64
 8   Adj. Open    1949 non-null   float64
 9   Adj. High    1949 non-null   float64
 10  Adj. Low     1949 non-null   float64
 11  Adj. Close   1949 non-null   float64
 12  Adj. Volume  1949 non-null   float64
dtypes: float64(12), object(1)
memory usage: 198.1+ KB
None


## Wiki Stocks Metadata

In [86]:
df = pd.read_csv('../data/wiki_stocks.csv')   #TODO: Add retrieval via NASDAQ Data Link / Quandl

# Save in HDFStore
print(df.info(null_counts=True))
with pd.HDFStore(DATA_STORE) as store:
    store.put('quandl/wiki/stocks', df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199 entries, 0 to 3198
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    3199 non-null   object
 1   name    3199 non-null   object
dtypes: object(2)
memory usage: 50.1+ KB
None


## S&P500 Constituents

In [87]:
# Retrieve SP500 constituents from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
df = pd.read_html(url, header=0)[0]
df.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


In [88]:
df.columns = ['ticker', 'name', 'sec_filings', 'gics_sector', 'gics_sub_industry',
              'location', 'first_added', 'cik', 'founded']
df = df.drop('sec_filings', axis=1).set_index('ticker')
display(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 503 entries, MMM to ZTS
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               503 non-null    object
 1   gics_sector        503 non-null    object
 2   gics_sub_industry  503 non-null    object
 3   location           503 non-null    object
 4   first_added        458 non-null    object
 5   cik                503 non-null    int64 
 6   founded            503 non-null    object
dtypes: int64(1), object(6)
memory usage: 31.4+ KB


None

,name,gics_sector,gics_sub_industry,location,first_added,cik,founded
ticker,,,,,,,
MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
ABMD,Abiomed,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


In [89]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('sp500/stocks', df)

## US-Equities Metadata

In [90]:
#TODO: Add docstring re: manual retrieval of dataset


In [91]:
# Read in csv file as dataframe
df = pd.read_csv('../data/nasdaq_equities.csv')
display(df.info())
display(df.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5148 entries, 0 to 5147
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      5147 non-null   object 
 1   Name        5148 non-null   object 
 2   Last Sale   5148 non-null   object 
 3   Net Change  5148 non-null   float64
 4   % Change    5148 non-null   object 
 5   Market Cap  5142 non-null   float64
 6   Country     5142 non-null   object 
 7   IPO Year    3439 non-null   float64
 8   Volume      5148 non-null   int64  
 9   Sector      4947 non-null   object 
 10  Industry    4947 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 442.5+ KB


None

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,AACG,ATA Creativity Global American Depositary Shares,$1.84,-0.03,-1.604%,58141402.0,China,2008.0,11058,Consumer Discretionary,Service to the Health Industry
1,AACI,Armada Acquisition Corp. I Common Stock,$9.92,0.00,0.00%,205438240.0,United States,2021.0,1950,Industrials,Consumer Electronics/Appliances
2,AACIU,Armada Acquisition Corp. I Unit,$10.035,0.00,0.00%,0.0,United States,2021.0,3,Industrials,Consumer Electronics/Appliances


In [92]:
# Rename 'Market Cap' to 'marketcap'
df.rename(columns={'Market Cap': 'marketcap'}, inplace=True)
df.head(2)

,Symbol,Name,Last Sale,Net Change,% Change,marketcap,Country,IPO Year,Volume,Sector,Industry
0,AACG,ATA Creativity Global American Depositary Shares,$1.84,-0.03,-1.604%,58141402.0,China,2008.0,11058,Consumer Discretionary,Service to the Health Industry
1,AACI,Armada Acquisition Corp. I Common Stock,$9.92,0.00,0.00%,205438240.0,United States,2021.0,1950,Industrials,Consumer Electronics/Appliances


In [93]:
mcap = df[['marketcap']].dropna()
print(mcap.info())
print(mcap.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5142 entries, 0 to 5147
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   marketcap  5142 non-null   float64
dtypes: float64(1)
memory usage: 80.3 KB
None
     marketcap
0   58141402.0
1  205438240.0
2          0.0
3          0.0
4  270696809.0


In [94]:
df['marketcap'] = mcap.marketcap
df.marketcap.describe(percentiles=np.arange(.1, 1, .1).round(1)).apply(lambda x: f'{int(x):,d}')

count                5,142
mean         3,899,693,405
std         54,222,711,343
min                      0
10%                      0
20%                      0
30%             11,221,869
40%             43,876,197
50%            109,960,453
60%            225,365,431
70%            396,854,033
80%            883,712,424
90%          2,812,218,431
max      2,479,066,246,600
Name: marketcap, dtype: object

In [95]:
# Preview dataframe
print(df.info())
print(df.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5148 entries, 0 to 5147
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      5147 non-null   object 
 1   Name        5148 non-null   object 
 2   Last Sale   5148 non-null   object 
 3   Net Change  5148 non-null   float64
 4   % Change    5148 non-null   object 
 5   marketcap   5142 non-null   float64
 6   Country     5142 non-null   object 
 7   IPO Year    3439 non-null   float64
 8   Volume      5148 non-null   int64  
 9   Sector      4947 non-null   object 
 10  Industry    4947 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 442.5+ KB
None
  Symbol                                              Name Last Sale  Net Change  % Change    marketcap        Country  IPO Year  Volume                  Sector                                    Industry
0   AACG  ATA Creativity Global American Depositary Shares     $1.84       -0.03   -1.

In [96]:
# Save df in HDFStore
with pd.HDFStore(DATA_STORE) as store:
    store.put('quandl/wiki/prices', df)